In [13]:
from collections import Counter
from typing import List, Tuple
import emoji
import json

def q2_time(file_path: str) -> List[Tuple[str, int]]:
    emoji_counter = Counter()

    # Leer todo el archivo en memoria
    with open(file_path, 'r') as file_data:
        lines = file_data.readlines()

    for line in lines:
        try:
            data = json.loads(line)
            content = data.get("content", "")
            # Contar emojis directamente si hay contenido
            if content:
                for value in emoji.analyze(content):
                    emoji_counter[value.chars] += 1
        except (json.JSONDecodeError, KeyError):
            # Manejar posibles errores en la carga de JSON
            continue

    # Rankear top 10 emojis más usados
    top_10_emojis = emoji_counter.most_common(10)
    return top_10_emojis

file_path = '/Users/juanignaciomagarinoscastro/Downloads/farmers-protest-tweets-2021-2-4.json'
q2_time(file_path)

[('🙏', 5049),
 ('😂', 3072),
 ('🚜', 2972),
 ('🌾', 2182),
 ('🇮🇳', 2086),
 ('🤣', 1668),
 ('✊', 1651),
 ('❤️', 1382),
 ('🙏🏻', 1317),
 ('💚', 1040)]